In [64]:
import sys
sys.path.append("../")
import saveload_utils as sl
import ferm_utils as feru
import csa_utils as csau
import var_utils as varu
import openfermion as of
import numpy as np
import scipy
import copy
import time 

alpha = 1
k = [[0,1],[2,3],[4,5],[6,7]]
# x_0 = np.ones(n)
# x = np.zeros(n)

mol = 'h4' 
tol = 1e-5
save = False
method_name = 'CAS-CAS'

# Get two-body tensor
Hf = sl.load_fermionic_hamiltonian(mol)
_, gs = of.linalg.get_ground_state(of.linalg.get_sparse_operator(Hf))
spin_orb = of.count_qubits(Hf)  
n = spin_orb
spatial_orb = spin_orb // 2
Htbt = feru.get_chemist_tbt(Hf, spin_orb, spin_orb = True)
one_body = varu.get_one_body_correction_from_tbt(Hf, feru.get_chemist_tbt(Hf))
# feru.get_one_body_terms(Hf)
# varu.get_one_body_correction_from_tbt(Hf, Htbt)


onebody_matrix = feru.get_obt(one_body, n = spin_orb, spin_orb = True)
onebody_tbt = feru.onebody_to_twobody(onebody_matrix)
# print(onebody_tbt.shape)
# print(Htbt.shape)

Htbt = np.add(Htbt, onebody_tbt)
upnum, casnum, pnum = csau.get_param_num(n, k = k, complex=False)
x0 = np.zeros(pnum * alpha)

cost = lambda x: csau.ct_decomp_cost(x, k, Htbt, alpha, complex=False)
jac = lambda x: csau.real_cas_grad(x, k, Htbt, alpha)
for i in range(alpha):
    x0[pnum * i + casnum: pnum *
        (i + 1)] = np.random.rand(upnum) * 2 * 100

print(scipy.optimize.check_grad(cost, jac, x0))
import time
start = time.time()
jac = jac(x0)
jac_time = time.time() - start
start = time.time()
numerical = scipy.optimize.approx_fprime(x0, cost)
numerical_time  = time.time() - start
print(numerical_time, jac_time)

8.594956436494277e-07
15.869324922561646 6.793452978134155
